In [1]:
import os
import io
import sys
import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd
import re
import subprocess
import numpy as np
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

sys.path.append('../src')

In [6]:
root_dir = '/' # TODO

databases_dir = {'C': 'C', 'EXPR': 'EXPR', 'VA': 'VA', 'VAE_VA': 'VAE', 'VAE_E': 'VAE'}

stats_path = 'logs/stats.csv'

results = {}
for k, db in databases_dir.items():
    results[k] = []
    exp_lists = os.listdir(os.path.join(root_dir, db))
    exp_lists.sort(key=lambda exp_name: '-'.join(exp_name.split('-')[-2:]), reverse=True)
    for exp_name in exp_lists:
        df = pd.read_csv(os.path.join(root_dir, db, exp_name, stats_path), sep=';')
        if 'C' in k:
            best_model = df.iloc[df['abaw_devel_f1'].idxmax()].to_dict()
        elif 'EXPR' in k:
            best_model = df.iloc[df['devel_f1'].idxmax()].to_dict()
        elif 'VAE_E' in k:
            best_model = df.iloc[df['expr_devel_f1'].idxmax()].to_dict()
        elif 'VAE_VA' in k:
            best_model = df.iloc[df['va_devel_va_score'].idxmax()].to_dict()
        elif 'VA' in k:
            best_model = df.iloc[df['devel_va_score'].idxmax()].to_dict()
            
        for name, v in best_model.items():
            if ('loss' in name) or ('epoch' in name) or ('experiment_name' in name) or ('va_score' in name) or ('v_score' in name) or ('a_score' in name):
                continue
            else:
                best_model[name] = v * 100

        results[k].append({**{'experiment_name': exp_name}, **best_model})

for k, db in results.items():
    pd.DataFrame(results[k]).to_csv('{0}_results.csv'.format(k.replace(os.sep, '_')), index=False)